In [71]:
import sys
import os
# requests allows you to send requests, without the need for manual labor
# https://2.python-requests.org/en/master/
import requests
# a toolkit for dissecting a document and extracting what you need
# https://www.crummy.com/software/BeautifulSoup/
from bs4 import BeautifulSoup

import pandas as pd

In [170]:
url = "http://www.adcon.rn.gov.br/ACERVO/sesap/DOC/DOC000000000228094.PDF"

# request to find the documents
pdf = requests.get(url)  

In [171]:
file_name = '{}'.format(
    "doc"
    .lstrip()  # remove initial space
    .lower()  # convert to lowcase
    .replace(' ', '_')
    .replace('/', '_')
    .replace('-', '_'))
open('{}.pdf'.format(file_name), 'wb').write(pdf.content)

351331

In [231]:
from tabula import read_pdf

# df = read_pdf("doc.pdf", pages=[8, 9, 10], stream=True)[0]
dfs = read_pdf("doc.pdf", stream=True, pages=[8, 9, 10])

In [221]:
dfs[2]

,municipio,suspeito,provavel,descartado,confirmado
0,NaN,SUSPEITO,PROVÁVEL,DESCARTADO,CONFIRMADO
1,RESIDÊNCIA,NaN,NaN,NaN,NaN
2,Frutuoso Gomes,-,-,1,-
3,Goianinha,3,-,1,-
4,Governado Dix-Sep,NaN,NaN,NaN,NaN
5,NaN,4,-,-,-
6,Rosado,NaN,NaN,NaN,NaN
7,Grossos,1,-,-,-
8,Guamaré,23,-,4,-
9,Ipanguaçu,1,-,-,-


In [230]:
# dfs[0].columns = ['municipio', 'suspeito', 'provavel', 'descartado', 'confirmado']
# df = dfs[0]
# df.drop(df.index[[0, 1]])

,municipio,suspeito,provavel,descartado,confirmado
2,Acari,10,-,-,-
3,Açu,29,-,4,-
4,Afonso Bezerra,1,-,-,-
5,Alto do Rodrigues,7,-,-,-
6,Angicos,8,-,1,-
7,Apodi,4,-,-,-
8,Areia Branca,2,-,-,-
9,Augusto Severo,1,-,-,-
10,Baía Formosa,2,-,1,-
11,Baraúna,21,-,-,-


In [220]:
#data = dfs[0].drop(df.index[[0, 1]])
#data.columns = ['municipio', 'suspeito', 'provavel', 'descartado', 'confirmado']

#for df in dfs[1:-1]:
#    df.columns = ['municipio', 'suspeito', 'provavel', 'descartado', 'confirmado']
#    df_ = df.drop(df.index[[0, 1]])
#    pd.concat([data, df_.reindex(data.index)])

In [360]:
df1 = dfs[0].drop(df.index[[0, 1]])
df2 = dfs[2].drop(df.index[[0, 1]])
df3 = dfs[3].drop(df.index[[0, 1]]) 


df1.columns = ['municipio', 'suspeito', 'provavel', 'descartado', 'confirmado']
df2.columns = ['municipio', 'suspeito', 'provavel', 'descartado', 'confirmado']
df3.columns = ['municipio', 'suspeito', 'provavel', 'descartado', 'confirmado'] 

In [433]:
data = pd.concat([df1, df2, df3])
data = data.reset_index(drop=True)
data

,municipio,suspeito,provavel,descartado,confirmado
0,Acari,10,-,-,-
1,Açu,29,-,4,-
2,Afonso Bezerra,1,-,-,-
3,Alto do Rodrigues,7,-,-,-
4,Angicos,8,-,1,-
...,...,...,...,...,...
111,Touros,3,-,-,-
112,Umarizal,1,-,-,-
113,Venha-Ver,1,-,-,-
114,Vera Cruz,-,-,1,-


In [270]:
data[data['municipio'] == "Natal"][['confirmado', 'suspeito']]

,confirmado,suspeito
33,22,524


In [271]:
df_old = pd.read_csv('https://raw.githubusercontent.com/leobezerra/covid19/master/data/rn_covid_19_boletins/03-25-2020.csv')  

In [272]:
df_old

,municipio,data,confirmado,suspeito,lat,lon
0,Acari,2020-03-25,0,6,-6.43395,-36.63959
1,Afonso Bezerra,2020-03-25,0,1,-5.50095,-36.50317
2,Alto do Rodrigues,2020-03-25,0,7,-5.29508,-36.76236
3,Angicos,2020-03-25,0,9,-5.67997,-36.60629
4,Apodi,2020-03-25,0,3,-5.66309,-37.79820
...,...,...,...,...,...,...
89,Tibau do Sul,2020-03-25,0,4,-6.18847,-35.09057
90,Touros,2020-03-25,0,3,-5.19887,-35.46050
91,Umarizal,2020-03-25,0,1,-5.99099,-37.81270
92,Venha-Ver,2020-03-25,0,1,-6.32000,-38.49093


In [162]:
for m in df_old.municipio:
    print(f"{m} -> {df_old[df_old['municipio'] == m][['confirmado', 'suspeito']]}")
    print(f"{m} -> {data[data['municipio'] == m][['confirmado', 'suspeito']]}")

Acari ->    confirmado  suspeito
0           0         6
Acari ->   confirmado suspeito
2          -        6
Afonso Bezerra ->    confirmado  suspeito
1           0         1
Afonso Bezerra ->   confirmado suspeito
4          -        1
Alto do Rodrigues ->    confirmado  suspeito
2           0         7
Alto do Rodrigues ->   confirmado suspeito
5          -        7
Angicos ->    confirmado  suspeito
3           0         9
Angicos ->   confirmado suspeito
6          -        9
Apodi ->    confirmado  suspeito
4           0         3
Apodi ->   confirmado suspeito
7          -        3
Areia Branca ->    confirmado  suspeito
5           0         4
Areia Branca ->   confirmado suspeito
8          -        4
Augusto Severo ->    confirmado  suspeito
6           0         1
Augusto Severo ->   confirmado suspeito
9          -        1
Açu ->    confirmado  suspeito
7           0        29
Açu ->   confirmado suspeito
3          -       29
Baraúna ->    confirmado  suspeito
8          

65           0         4
Pau dos Ferros -> Empty DataFrame
Columns: [confirmado, suspeito]
Index: []
Pedro Velho ->     confirmado  suspeito
66           0         2
Pedro Velho -> Empty DataFrame
Columns: [confirmado, suspeito]
Index: []
Pendências ->     confirmado  suspeito
67           0         1
Pendências -> Empty DataFrame
Columns: [confirmado, suspeito]
Index: []
Pilões ->     confirmado  suspeito
68           0         1
Pilões -> Empty DataFrame
Columns: [confirmado, suspeito]
Index: []
Rafael Fernandes ->     confirmado  suspeito
69           0         1
Rafael Fernandes -> Empty DataFrame
Columns: [confirmado, suspeito]
Index: []
Riachuelo ->     confirmado  suspeito
70           0         1
Riachuelo -> Empty DataFrame
Columns: [confirmado, suspeito]
Index: []
Rio do Fogo ->     confirmado  suspeito
71           0         3
Rio do Fogo -> Empty DataFrame
Columns: [confirmado, suspeito]
Index: []
Ruy Barbosa ->     confirmado  suspeito
72           0         1
Ruy Barbosa 

In [273]:
df_old[df_old['municipio'] == "Natal"][['confirmado', 'suspeito']]

,confirmado,suspeito
56,13,532


In [274]:
data[data['municipio'] == "Natal"][['confirmado', 'suspeito']]

,confirmado,suspeito
33,22,524


In [397]:
data[data['municipio'].isnull()]

,municipio,suspeito,provavel,descartado,confirmado
34,NaN,4,-,-,-
66,NaN,1,-,-,-
90,NaN,1,-,-,-
93,NaN,21,-,7,-
98,NaN,4,-,1,-
103,NaN,1,-,2,-


In [398]:
data[data.isnull().any(1)]

,municipio,suspeito,provavel,descartado,confirmado
33,Governado Dix-Sep,NaN,NaN,NaN,NaN
34,NaN,4,-,-,-
35,Rosado,NaN,NaN,NaN,NaN
51,Lucrécia,1,NaN,NaN,NaN
65,Olho-d'Água do,NaN,NaN,NaN,NaN
66,NaN,1,-,-,-
67,Borges,NaN,NaN,NaN,NaN
68,Parazinho,NaN,-,1,-
89,São Francisco do,NaN,NaN,NaN,NaN
90,NaN,1,-,-,-


In [396]:
data[data['municipio'].isnull()].iloc[0, [0, 1, 2, 3, 4]].name

34

In [370]:
data[data.isnull().any(1)].iloc[[0, 2], 0]

4    Governado Dix-Sep
6               Rosado
Name: municipio, dtype: object

In [371]:
data[data.isnull().any(1)].iloc[0, 0]

'Governado Dix-Sep'

In [407]:
data = data.reset_index(drop=True)

In [409]:
data

,municipio,suspeito,provavel,descartado,confirmado
0,Acari,10,-,-,-
1,Açu,29,-,4,-
2,Afonso Bezerra,1,-,-,-
3,Alto do Rodrigues,7,-,-,-
4,Angicos,8,-,1,-
...,...,...,...,...,...
111,Touros,3,-,-,-
112,Umarizal,1,-,-,-
113,Venha-Ver,1,-,-,-
114,Vera Cruz,-,-,1,-


In [393]:
# data.at[4, 'municipio'] = ' '.join(data[data.isnull().any(1)].iloc[[0, 2], 0])

In [ ]:
# data[data.isnull().any(1)].at[4, 'municipio']

In [399]:
data[data['municipio'].isnull()]
# index = row.name

,municipio,suspeito,provavel,descartado,confirmado
34,NaN,4,-,-,-
66,NaN,1,-,-,-
90,NaN,1,-,-,-
93,NaN,21,-,7,-
98,NaN,4,-,1,-
103,NaN,1,-,2,-


In [420]:
data

,municipio,suspeito,provavel,descartado,confirmado
0,Acari,10,-,-,-
1,Açu,29,-,4,-
2,Afonso Bezerra,1,-,-,-
3,Alto do Rodrigues,7,-,-,-
4,Angicos,8,-,1,-
...,...,...,...,...,...
111,Touros,3,-,-,-
112,Umarizal,1,-,-,-
113,Venha-Ver,1,-,-,-
114,Vera Cruz,-,-,1,-


In [434]:
drop_lines = []
for index, row in data[data['municipio'].isnull()].iterrows():
    print(' '.join(data.iloc[[index-1, index+1], 0]))
    
    data.at[index-1, 'municipio'] = ' '.join(data.iloc[[index-1, index+1], 0])
    data.at[index-1, 'suspeito'] = data.iloc[index, 1]
    data.at[index-1, 'provavel'] = data.iloc[index, 2]
    data.at[index-1, 'confirmado'] = data.iloc[index, 3]
    drop_lines.extend([index, index+1])

new_data = data.drop(drop_lines)

Governado Dix-Sep Rosado
Olho-d'Água do Borges
São Francisco do Oeste
São Gonçalo do Amarante
São José do Campestre
São Miguel do Gostoso


In [437]:
new_data[new_data["municipio"] == "Governado Dix-Sep Rosado"]

,municipio,suspeito,provavel,descartado,confirmado
33,Governado Dix-Sep Rosado,4,-,NaN,-


In [438]:
new_data[new_data.isnull().any(1)]

,municipio,suspeito,provavel,descartado,confirmado
33,Governado Dix-Sep Rosado,4,-,NaN,-
51,Lucrécia,1,NaN,NaN,NaN
65,Olho-d'Água do Borges,1,-,NaN,-
68,Parazinho,NaN,-,1,-
89,São Francisco do Oeste,1,-,NaN,-
92,São Gonçalo do Amarante,21,-,NaN,7
97,São José do Campestre,4,-,NaN,1
102,São Miguel do Gostoso,1,-,NaN,2


In [446]:
for index, row in new_data.iterrows():
    print(row["municipio"])

Acari
Açu
Afonso Bezerra
Alto do Rodrigues
Angicos
Apodi
Areia Branca
Augusto Severo
Baía Formosa
Baraúna
Barcelona
Bodó
Bom Jesus
Brejinho
Caiçara do Norte
Caicó
Canguaretama
Caraúbas
Carnaubais
Ceará-Mirim
Cerro Corá
Coronel João Pessoa
Cruzeta
Currais Novos
Doutor Severiano
Parnamirim
Espírito Santo
Extremoz
Felipe Guerra
Fernando Pedroza
Florânia
Frutuoso Gomes
Goianinha
Governado Dix-Sep Rosado
Grossos
Guamaré
Ipanguaçu
Ipueira
Itajá
Januário Cicco
Japi
Jardim do Seridó
João Câmara
Jucurutu
Jundiá
Lagoa d’Anta
Lagoa de Velhos
Lagoa Salgada
Lajes
Lucrécia
Luís Gomes
Macaíba
Macau
Major Sales
Marcelino Vieira
Maxaranguape
Montanhas
Monte Alegre
Monte das Gameleiras
Mossoró
Natal
Nísia Floresta
Nova Cruz
Olho-d'Água do Borges
Parazinho
Parelhas
Rio do Fogo
Riacho da Cruz
Passa e Fica
Passagem
Patu
Santa Maria
Pau dos Ferros
Pedro Velho
Pendências
Pilões
Serra Caiada
Rafael Fernandes
Riachuelo
Tibau
Ruy Barbosa
Santa Cruz
Santana dos Matos
Santo Antônio
São Fernando
São Francisco do O

In [448]:
new_data[new_data["municipio"] == "Natal"]

,municipio,suspeito,provavel,descartado,confirmado
62,Natal,524,-,181,22
